<a href="https://colab.research.google.com/github/vis7/recommendation_system/blob/main/tensorflow_recommender_tutorial/movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installation
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 4.0 MB 16.0 MB/s 


In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
# loading data
ratings = tfds.load('movielens/100k-ratings', split='train')
movies = tfds.load('movielens/100k-movies', split='train')

# selecting basic feature
ratings = ratings.map(lambda x:{
    'movie_title': x['movie_title'],
    'user_id': x['user_id']
})
movies = movies.map(lambda x: x['movie_title'])


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling movielens-train.tfrecord...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [ ]:
# printing basic details about dataset

In [ ]:
# creating integer indices from string lookup
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x:x['user_id']))

movie_title_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_title_vocabulary.adapt(movies)

In [ ]:
# definging a model
class MovieLensModel(tfrs.Model):
  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval
  ):
    super().__init__()
    self.user_model = user_model
    self.movie_model = movie_model
    self.task = task
  
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    user_embeddings = self.user_model(features['user_id'])
    movie_embeddings = self.movie_model(features['movie_title'])

    return self.task(user_embeddings, movie_embeddings)


In [ ]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_title_vocabulary,
    tf.keras.layers.Embedding(movie_title_vocabulary.vocab_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [ ]:
# creating retrieval model
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [ ]:
# train for 3 epochs
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 8.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0050 - factorized_top_k/top_50_categorical_accuracy: 0.0443 - factorized_top_k/top_100_categorical_accuracy: 0.1030 - loss: 33090.8260 - regularization_loss: 0.0000e+00 - total_loss: 33090.8260
Epoch 2/3
25/25 [==============================] - 27s 1s/step - factorized_top_k/top_1_categorical_accuracy: 1.7000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0145 - factorized_top_k/top_50_categorical_accuracy: 0.1055 - factorized_top_k/top_100_categorical_accuracy: 0.2107 - loss: 31013.7435 - regularization_loss: 0.0000e+00 - total_loss: 31013.7435
Epoch 3/3
25/25 [==============================] - 27s 1s/step - factorized_top_k/top_1_categorical_accuracy: 6.9000e-04 - factorized_top_k/top_5_categorical_accu

In [ ]:
# use brute-force search to set up retrival using the trained representations
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

In [ ]:
# getting some recommendations
_, titles = index(np.array(["42"]))
print(f"top k recommendations for user 42: {titles[0,:3]}")

top k recommendations for user 42: [b'Rent-a-Kid (1995)' b'Just Cause (1995)' b'House Arrest (1996)']
